In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import matplotlib.pyplot as plt
import numpy as np
import torch
import xarray as xr

In [ ]:
from crims2s.dataset import S2SDataset, TransformedDataset, netcdf_file_groups

from crims2s.transform import AddLatLonFeature, AddBiweeklyDimTransform, CompositeTransform, LongitudeRoll, MembersSubsetTransform

In [ ]:
with hydra.initialize_config_module('crims2s.training.conf'):
    cfg = hydra.compose('config', overrides=['experiment/dataset=lean', 'experiment/transform=geography'])

In [ ]:
t = hydra.utils.instantiate(cfg.experiment.transform)

In [ ]:
t

In [ ]:
d = S2SDataset(cfg.experiment.dataset.dataset_dir)

In [ ]:
d = TransformedDataset(d, t)

In [ ]:
d[0]["features"]

In [ ]:
plt.imshow(d[0]['model_parameters_t2m_mu'][0])

In [ ]:
MembersSubsetTransform(1)(d[0])['features']

In [ ]:
features = LongitudeRoll()(AddLatLonFeatureXarray()(d[0]))['features']

In [ ]:
features.sel(variable='lon').isel(lead_time=0, realization=0).features.plot()

In [ ]:
features['features']

In [ ]:
features.assign(lon=features.longitude)

In [ ]:
d[0].keys()

In [ ]:
rolled = LongitudeRoll()(d[0])

In [ ]:
rolled['features']

In [ ]:
netcdf_file_groups(d.dataset.files[0])

In [ ]:
sample = d[0]

In [ ]:
sample.keys()

In [ ]:
sample['eccc_available']

In [ ]:
plt.imshow(sample['eccc_parameters_tp_sigma'][0])

In [ ]:
d[0].keys()

In [ ]:
d[0]['features'].lead_time

In [ ]:
first_example = d[0]

In [ ]:
first_example['features_features'].shape

In [ ]:
lons = first_example['features_features'][..., -1]

In [ ]:
lats = first_example['features_features'][..., -2]

In [ ]:
first_example['features_features'].shape

In [ ]:
lons.max()

In [ ]:
plt.imshow(lons[0,0,:, :, 0])

In [ ]:
sorted(first_example.keys())

In [ ]:
first_example['latitude']

In [ ]:
latitude = first_example['latitude']

In [ ]:
weights = torch.where(latitude > -60.0, torch.cos(torch.deg2rad(torch.abs(latitude))), 0.0)

In [ ]:
weights

In [ ]:
plt.imshow(first_example['dry_mask_tp'][0])